# Flux Balance Analysis of Protein Lactate Dehydrogenase

In [325]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import cobra as cb

Loading the Model


We will use the E. coli genome-scale metabolic model. The E. coli model has been curated for over 20 years through several publications and can be downloaded from the BiGG Database. We will use the iML1515.xml model which is one of the latest versions, published in 2017. We will load the model below using COBRApy.

In [326]:
# load model
model = cb.io.read_sbml_model('iML1515.xml')

In [327]:
model

Name,iML1515
Memory address,215f1329120
Number of metabolites,1877
Number of reactions,2712
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


Defining the Environment

A typical flux balance analysis will start by defining the environment for the simulation. This is done by changing the bounds on the "exchange" reactions. Exchange reactions are reactions in the metabolic network that import or export metabolites into the system. By default they are exporters, so a positive flux is exporting the metabolite and a negative flux is importing. We set the lower bound on select exchanges to a negative value to define envirnmental metabolites that can be imported in the simulation.

In the code below, we will start by allowing export of all metabolites and import of no metabolites, to set a blank slate. Then we will allow import of all metabolites in a glucose minimal medium (a typical growth medium used for E. coli). We will also print out the names of the glucose minimal medium to see what we are adding into the simulation environment.

In [328]:
# Define environment

# Set a blank slate
for ex in model.exchanges:
    ex.lower_bound = 0
    ex.upper_bound = 1000
    
# glucose minimal medium
# glc_min_med = ['EX_pi_e','EX_co2_e','EX_fe3_e','EX_h_e','EX_mn2_e','EX_fe2_e','EX_glc__D_e','EX_zn2_e',
#                'EX_mg2_e','EX_ca2_e','EX_ni2_e','EX_cu2_e','EX_sel_e','EX_cobalt2_e','EX_h2o_e','EX_mobd_e',
#                'EX_so4_e','EX_nh4_e','EX_k_e','EX_na1_e','EX_cl_e','EX_o2_e','EX_tungs_e','EX_slnt_e']
glc_min_med = ['EX_pi_e','EX_co2_e','EX_fe3_e','EX_h_e','EX_mn2_e','EX_fe2_e','EX_glc__D_e','EX_zn2_e',
               'EX_mg2_e','EX_ca2_e','EX_ni2_e','EX_cu2_e','EX_sel_e','EX_cobalt2_e','EX_h2o_e','EX_mobd_e',
               'EX_so4_e','EX_nh4_e','EX_k_e','EX_na1_e','EX_cl_e','EX_tungs_e','EX_slnt_e']



for ex_id in glc_min_med:
    model.exchanges.get_by_id(ex_id).lower_bound = -1000
    print(model.exchanges.get_by_id(ex_id).id, model.exchanges.get_by_id(ex_id).name)

# set glucose exchange lower bound to -18.5
model.exchanges.get_by_id('EX_glc__D_e').lower_bound = -18.5

EX_pi_e Phosphate exchange
EX_co2_e CO2 exchange
EX_fe3_e Fe3+ exchange
EX_h_e H+ exchange
EX_mn2_e Mn2+ exchange
EX_fe2_e Fe2+ exchange
EX_glc__D_e D-Glucose exchange
EX_zn2_e Zinc exchange
EX_mg2_e Mg exchange
EX_ca2_e Calcium exchange
EX_ni2_e Ni2+ exchange
EX_cu2_e Cu2+ exchange
EX_sel_e Selenate exchange
EX_cobalt2_e Co2+ exchange
EX_h2o_e H2O exchange
EX_mobd_e Molybdate exchange
EX_so4_e Sulfate exchange
EX_nh4_e Ammonia exchange
EX_k_e K+ exchange
EX_na1_e Sodium exchange
EX_cl_e Chloride exchange
EX_tungs_e Tungstate exchange
EX_slnt_e Selenite exchange


Now that we have defined the environment for our simulation we can simulate the flux through the metabolic network. To do this, flux balance analysis will typically optimize a reaction called the biomass reaction. The biomass reaction consumes all of the essential building blocks for the cell (amino acids, nucleotides, lipids, etc...). Optimizing this reaction makes the assumption that the metabolic network flux is optimally producing the biomass precursors. We will implement the simulation by optimizing biomass flux below:

In [329]:
# This code prints out the metabolites that are consumed by the biomass reaction
# It is not necessary to run but gives some insight into the model assumptions

model.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
bio_mets = list(model.reactions.get_by_id(str(model.objective.expression).split(' ')[0].split('*')[1]).metabolites.keys())
bio_stoich = list(model.reactions.get_by_id(str(model.objective.expression).split(' ')[0].split('*')[1]).metabolites.values())
bio_mets_c = []
for i in range(len(bio_mets)):
    if bio_stoich[i] < 0:
        bio_mets_c.append(bio_mets[i].name)
n_M = len(bio_mets_c)
bio_mets_c

['10-Formyltetrahydrofolate',
 '[2Fe-2S] iron-sulfur cluster',
 '2-Octaprenyl-6-hydroxyphenol',
 '[4Fe-4S] iron-sulfur cluster',
 'L-Alanine',
 'S-Adenosyl-L-methionine',
 'L-Arginine',
 'L-Asparagine',
 'L-Aspartate',
 'ATP C10H12N5O13P3',
 'Biotin',
 'Calcium',
 'Chloride',
 'Coenzyme A',
 'Co2+',
 'CTP C9H12N3O14P3',
 'Copper',
 'L-Cysteine',
 'DATP C10H12N5O12P3',
 'DCTP C9H12N3O13P3',
 'DGTP C10H12N5O13P3',
 'DTTP C10H13N2O14P3',
 'Flavin adenine dinucleotide oxidized',
 'Fe2+ mitochondria',
 'Iron (Fe3+)',
 'L-Glutamine',
 'L-Glutamate',
 'Glycine',
 'GTP C10H12N5O14P3',
 'H2O H2O',
 'L-Histidine',
 'L-Isoleucine',
 'Potassium',
 'KDO(2)-lipid IV(A)',
 'L-Leucine',
 'L-Lysine',
 'L-Methionine',
 'Magnesium',
 '5,10-Methylenetetrahydrofolate',
 'Manganese',
 'Molybdate',
 'Two disacharide linked murein units, pentapeptide crosslinked tetrapeptide (A2pm->D-ala) (middle of chain)',
 'Nicotinamide adenine dinucleotide',
 'Nicotinamide adenine dinucleotide phosphate',
 'Ammonium',
 'N

In [330]:
# Simulate Flux
model.objective = 'BIOMASS_Ec_iML1515_core_75p37M'
solution = model.optimize()
print(solution.objective_value)

0.3420889409686897


Knocking Out Genes

Genome-scale metabolic models contain a boolean mapping from genes to metabolic reactions. With this information we can simulate gene knockouts and their effect on metabolism. Let's try knocking out some genes and see what happens.

We will knock out the gene HisD which is used for the production of the amino acid histidine in E. coli. We will then simulate growth with FBA to see if this gene was essential.

In [333]:
# Reset the environment
# Set a blank slate
for ex in model.exchanges:
    ex.lower_bound = 0
    ex.upper_bound = 1000
    
# glucose minimal medium
glc_min_med = ['EX_pi_e','EX_co2_e','EX_fe3_e','EX_h_e','EX_mn2_e','EX_fe2_e','EX_glc__D_e','EX_zn2_e',
               'EX_mg2_e','EX_ca2_e','EX_ni2_e','EX_cu2_e','EX_sel_e','EX_cobalt2_e','EX_h2o_e','EX_mobd_e',
               'EX_so4_e','EX_nh4_e','EX_k_e','EX_na1_e','EX_cl_e','EX_o2_e','EX_tungs_e','EX_slnt_e']

for ex_id in glc_min_med:
    model.exchanges.get_by_id(ex_id).lower_bound = -1000

# set glucose exchange lower bound to -10
model.exchanges.get_by_id('EX_glc__D_e').lower_bound = -18.5

# Simulate Gene Deletion
# Delete Gene ldhA which is essential for lactate metabolism in E. coli
with model:
    model.genes.get_by_id('b1380').knock_out()
    solution2 = model.optimize()
    print('Biomass Flux with LDH KO:', solution2.objective_value)

Biomass Flux with LDH KO: 1.6443791812413442


Double Gene Knockouts

In general, metabolic models allow us to simulate metabolic phenotypes at a much higher scale than what can be feasible accomplished through experiments. For example, we can simulate any combination of gene knockouts. For an organism with 1516 metabolic genes that is  $2^{1516}$
  possible experiments (actually exploring this whole space is even beyond what we can simulate in a reasonable amount of compute time). Experimentally, researchers have measured the effects of double gene knockouts in various organisms. Occasionally, a double gene knockout will have a phenotype that is unexpected based on the single gene knock outs. For example, neither gene is essential individually but when they are both knocked out they become essential. This unexpected effect is generally termed "epistasis" and the specific example given here is known as "synthetic lethality". Let's try simulating some random double gene knockouts to see what the model predicts.

In [335]:
    
# Simulate Gene Deletions
with model:
    model.genes.get_by_id('b1380').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b3187').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ispB', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b1380').knock_out()
    model.genes.get_by_id('b3187').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA and ispB', solution1.objective_value)

Biomass Flux Knock out ldhA 1.6443791812413449
Biomass Flux Knock out ispB 1.4146894476415339e-15
Biomass Flux Knock out ldhA and ispB 0.0


In [336]:
    
# Simulate Gene Deletions
with model:
    model.genes.get_by_id('b1380').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b1208').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ispE', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b1380').knock_out()
    model.genes.get_by_id('b1208').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA and ispE', solution1.objective_value)

Biomass Flux Knock out ldhA 1.6443791812413466
Biomass Flux Knock out ispE 6.859100352201377e-16
Biomass Flux Knock out ldhA and ispE 0.0


In [337]:

# Simulate Gene Deletions
with model:
    model.genes.get_by_id('b1380').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b3972').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out murB', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b1380').knock_out()
    model.genes.get_by_id('b3972').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA and murB', solution1.objective_value)

Biomass Flux Knock out ldhA 1.644379181241333
Biomass Flux Knock out murB 1.1574731844339823e-15
Biomass Flux Knock out ldhA and murB 0.0


In [338]:
  
# Simulate Gene Deletions
with model:
    model.genes.get_by_id('b1380').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b0085').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out murE', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b1380').knock_out()
    model.genes.get_by_id('b0085').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA and murE', solution1.objective_value)

Biomass Flux Knock out ldhA 1.6443791812413342
Biomass Flux Knock out murE 6.4304065801887915e-16
Biomass Flux Knock out ldhA and murE 0.0


In [340]:
    
# Simulate Gene Deletions
with model:
    model.genes.get_by_id('b1380').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b0142').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out folK', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b1380').knock_out()
    model.genes.get_by_id('b0142').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA and folK', solution1.objective_value)

Biomass Flux Knock out ldhA 1.6443791812413615
Biomass Flux Knock out folK 2.7522140163208027e-13
Biomass Flux Knock out ldhA and folK -2.0902614464316067e-26


In [341]:
    
# Simulate Gene Deletions
with model:
    model.genes.get_by_id('b1380').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b2153').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out folE', solution1.objective_value)
    
with model:
    model.genes.get_by_id('b1380').knock_out()
    model.genes.get_by_id('b2153').knock_out()
    solution1 = model.optimize()
    print('Biomass Flux Knock out ldhA and folE', solution1.objective_value)

Biomass Flux Knock out ldhA 1.6443791812413615
Biomass Flux Knock out folE 8.002426642158943e-13
Biomass Flux Knock out ldhA and folE 0.0


The models above simiulates growth when we Knock them both genes out individually but simiulates no growth when both are knocked out together, this is because neither gene is essential individually but when they are both knocked out they become essential. This unexpected effect is generally termed "epistasis" and is known as "synthetic lethality".